In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fft2, ifft2, fftfreq
from scipy.integrate import solve_ivp
from scipy.linalg import toeplitz
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# AMATH 581 Homework 5
## Eric Leonard, ericcl@uw.edu

## Problem 1
Consider the $\lambda − \omega$ reaction-diffusion system

$$U_t = \lambda(A)U − \omega(A)V + D_1\nabla^2U$$

$$V_t = \omega(A)U + \lambda(A)V + D_2\nabla^2V$$

where $A^2 = U^2 + V^2$ and $\nabla^2 = \partial^2_x + \partial^2_y$.

**Boundary Conditions**: Consider the two boundary conditions in the $x$− and $y$−directions:
- Periodic
- No flux: $\partial U / \partial n = \partial V / \partial n = 0$ on the boundaries

**Numerical Integration Procedure**: The following numerical integration procedures are to be investigated and compared.
- For the periodic boundaries, transform the right-hand with FFTs
- For the no flux boundaries, use the Chebychev polynomials

You can advance the solution in time using ```solve_ivp``` with RK45

Initial Conditions Start with spiral initial conditions in $U$ and $V$.

```python
X,Y = np.meshgrid(x,y)
m = 1 # number of spirals
i = complex(0,1)
u = np.tanh(np.sqrt(X**2 + Y**2)) * np.cos(m * np.angle(X + i*Y) - (np.sqrt(X**2 + Y*2)))
v = np.tanh(np.sqrt(X**2 + Y**2)) * np.sin(m * np.angle(X + i*Y) - (np.sqrt(X**2 + Y**2)))
```

Consider the specific $\lambda − \omega$ system:

$$\lambda(A) = 1 − A^2$$
$$\omega(A) = −\beta A^2$$

Look to construct one- and two-armed spirals for this system. Also investigate when the solutions become unstable and “chaotic” in nature. Investigate the system for all three boundary conditions. Note $\beta > 0$ and further consider the diffusion to be not too large, but big enough to kill off Gibbs phenomena at the boundary, i.e. $D1 = D2 = 0.1$.

**ANSWERS**: With $x, y \in [−10, 10]$, $n = 64$, $\beta = 1$, $D_1 = D_2 = 0.1$, $m = 1$, $tspan = 0 : 0.5 : 4$ and $u$ stacked on top of $v$, write out the solution of your numerical evolution from solve_ivp with RK45 with periodic boundaries as A1. (NOTE: your solution will be in the Fourier domain when your write it out.)

**ANSWERS**: With $x, y \in [−10, 10]$, $n = 30$, $\beta = 1$, $D_1 = D_2 = 0.1$, $m = 1$, $tspan = 0 : 0.5 : 4$ and $u$ stacked on top of $v$, write out the solution of your numerical evolution from solve_ivp with RK45 with no-flux boundaries as A2. (NOTE: be sure to remember that you have to rescale the problem to $-1$ to $1$ for chebyshev polynomials)

In [2]:
# Parameters
L = 10  # domain size
n = 64  # grid points
beta = 1  # parameter beta
D1 = D2 = 0.1  # diffusion coefficients
t_span = (0, 4)  # time span
t_eval = np.arange(0, 4.5, 0.5)  # evaluation times

# Spatial grid
x = np.linspace(-L, L, n+1)
y = np.linspace(-L, L, n+1)
X, Y = np.meshgrid(x[:-1], y[:-1])

# Wave numbers for FFT
k = 2 * np.pi * fftfreq(n, d=(2*L/n))
kx, ky = np.meshgrid(k, k)
laplacian_fft = -(kx**2 + ky**2)

# Initial conditions
m = 1  # number of spirals

In [3]:
U0 = np.tanh(np.sqrt(X**2 + Y**2)) * np.cos(m * np.angle(X + 1j*Y) - np.sqrt(X**2 + Y**2))
V0 = np.tanh(np.sqrt(X**2 + Y**2)) * np.sin(m * np.angle(X + 1j*Y) - np.sqrt(X**2 + Y**2))
UV0 = np.concatenate([fft2(U0).flatten(), fft2(V0).flatten()])

# Reaction-diffusion system
def reaction_diffusion(t, UV):
    U_fft = UV[:n**2].reshape((n, n))
    V_fft = UV[n**2:].reshape((n, n))
    U, V = ifft2(U_fft), ifft2(V_fft)
    A2 = U**2 + V**2
    lambda_A = 1 - A2
    omega_A = -beta * A2

    U_lap = laplacian_fft * U_fft
    V_lap = laplacian_fft * V_fft
    
    U_t = fft2(lambda_A * U - omega_A * V) + D1 * U_lap
    V_t = fft2(omega_A * U + lambda_A * V) + D2 * V_lap

    return np.concatenate([U_t.flatten(), V_t.flatten()])

# Solve the system
sol = solve_ivp(reaction_diffusion, t_span, UV0, method='RK45', t_eval=t_eval)

# Extract the solution in Fourier domain (Periodic Boundaries)
U_sol = sol.y[:n**2].reshape(n, n, len(t_eval))
V_sol = sol.y[n**2:].reshape(n, n, len(t_eval))

# Output Fourier domain solutions as A1
A1 = sol.y
print(A1)

[[ 24.94003847+0.00000000e+00j  12.73268299+2.57334699e-15j
   -1.38095598-2.40406635e-15j ... -64.02389647-6.05615274e-14j
  -67.76356741-5.93935290e-14j -61.18058974-4.13572632e-14j]
 [-18.55666362-5.81663109e+01j -42.51586944-4.69129224e+01j
  -60.80795253-2.57480390e+01j ... -26.39439597+1.13082890e+02j
    6.86544434+1.23000456e+02j  41.4436393 +1.10055312e+02j]
 [-16.04755868+3.28279829e+01j -22.03971648-4.57977740e+01j
  -23.23089505-1.04141716e+02j ... -25.03391682-9.26527314e+01j
  -29.2936105 -4.09594873e+01j -31.3712619 +1.56986891e+01j]
 ...
 [ 24.73021466-5.66774723e+02j  34.94179045-3.31372917e+02j
   38.82924248-4.97842318e+01j ...   4.99619196+6.02396295e+02j
   -9.93322885+4.90736906e+02j -25.6299042 +2.81792021e+02j]
 [ 25.33720124-3.61633792e+02j  43.00958768-4.53711746e+02j
   51.93221654-4.47841562e+02j ... -30.76392977+2.66442187e+02j
  -58.45411318+4.29165358e+02j -74.0191717 +5.05315322e+02j]
 [ -6.4753501 +3.96245454e+01j  15.86720969-5.83358549e+01j
   37.7389

In [4]:
### ANIMATION ###

# Function to create an animation of U and V
def animate_solution(U_sol, V_sol, x, t_eval, title_suffix):
    fig, ax = plt.subplots(1, 2, figsize=(14, 6))
    extent = (-L, L, -L, L)

    # Initialize plots
    print(U_sol.shape)
    im_U = ax[0].imshow(U_sol[:, :, 0], extent=extent, origin='lower', aspect='auto', cmap='RdBu')
    ax[0].set_title(f'U (t={t_eval[0]:.2f}) {title_suffix}')
    ax[0].set_xlabel('x')
    ax[0].set_ylabel('y')
    fig.colorbar(im_U, ax=ax[0], label='U')

    im_V = ax[1].imshow(V_sol[:, :, 0], extent=extent, origin='lower', aspect='auto', cmap='RdBu')
    ax[1].set_title(f'V (t={t_eval[0]:.2f}) {title_suffix}')
    ax[1].set_xlabel('x')
    fig.colorbar(im_V, ax=ax[1], label='V')

    # Update function for animation
    def update(frame):
        im_U.set_data(U_sol[:, :, frame])
        ax[0].set_title(f'U (t={t_eval[frame]:.2f}) {title_suffix}')
        im_V.set_data(V_sol[:, :, frame])
        ax[1].set_title(f'V (t={t_eval[frame]:.2f}) {title_suffix}')
        return im_U, im_V

    video_duration = t_eval[-1] * 1000 # video duration in miliseconds
    interval = video_duration / len(t_eval)
    ani = FuncAnimation(fig, update, frames=len(t_eval), interval=interval, blit=True)
    plt.close(fig)
    return ani

In [5]:
### ANIMATION ###

t_span_movie = (0, 10)  # time span
t_eval_movie = np.arange(0, 10.1, 0.1)  # evaluation times

# Solve the system
sol = solve_ivp(reaction_diffusion, t_span_movie, UV0, method='RK45', t_eval=t_eval_movie)

# Extract the solution in Fourier domain (Periodic Boundaries)
U_sol_fft = sol.y[:n**2].reshape((n, n, len(t_eval_movie)))
V_sol_fft = sol.y[n**2:].reshape((n, n, len(t_eval_movie)))
U_sol = np.zeros((n,n,len(t_eval_movie)))
V_sol = np.zeros((n,n,len(t_eval_movie)))
for i in range(len(t_eval_movie)):
    U_sol[:, :, i] = np.real(ifft2(U_sol_fft[:, : , i]))
    V_sol[:, :, i] = np.real(ifft2(V_sol_fft[:, : , i]))

# Create animation for periodic boundary results
ani_periodic = animate_solution(np.real(U_sol), np.real(V_sol), x, t_eval_movie, "(Periodic Boundaries)")
ani_periodic.save("fft_reaction_diffusion.mp4")
# Display animation
HTML(ani_periodic.to_html5_video())

(64, 64, 101)


In [6]:
# Rescale [-1, 1] grid to [-L, L]
def rescale_chebyshev_grid(x, L):
    return L * x

def cheb(N):
    n = np.arange(0,N+1)
    x = rescale_chebyshev_grid(np.cos(np.pi*n/N).reshape(N+1,1), L)
    c = (np.hstack(( [2.], np.ones(N-1), [2.]))*(-1)**n).reshape(N+1,1)
    X = np.tile(x,(1,N+1))
    dX = X - X.T
    D = (c @ (1 / c.T)) / (dX + np.eye(N+1))
    D -= np.diag(np.sum(D.T,axis=0))
    return D, x.reshape(N+1)
# Parameters for Chebyshev method
n_chebyshev = 30
D_cheb, x_cheb = cheb(n_chebyshev)
D_cheb[0, :] = 0
D_cheb[-1,:] = 0

# Laplacian using Chebyshev differentiation
D_xx = D_cheb @ D_cheb
laplacian_cheb = np.kron(np.eye(n_chebyshev + 1), D_xx) + np.kron(D_xx, np.eye(n_chebyshev + 1))

X_cheb, Y_cheb = np.meshgrid(x_cheb, x_cheb)

# Initial conditions for no-flux boundaries
U0_cheb = np.tanh(np.sqrt(X_cheb**2 + Y_cheb**2)) * np.cos(m * np.angle(X_cheb + 1j*Y_cheb) - np.sqrt(X_cheb**2 + Y_cheb**2))
V0_cheb = np.tanh(np.sqrt(X_cheb**2 + Y_cheb**2)) * np.sin(m * np.angle(X_cheb + 1j*Y_cheb) - np.sqrt(X_cheb**2 + Y_cheb**2))
UV0_cheb = np.concatenate([U0_cheb.flatten(), V0_cheb.flatten()])

# Reaction-diffusion system with Chebyshev differentiation
def reaction_diffusion_cheb(t, UV):
    U = UV[:(n_chebyshev + 1)**2]
    V = UV[(n_chebyshev + 1)**2:]
    A2 = U**2 + V**2
    lambda_A = 1 - A2
    omega_A = -beta * A2

    # Time derivatives
    U_t = lambda_A * U - omega_A * V + D1 * laplacian_cheb @ U
    V_t = omega_A * U + lambda_A * V + D2 * laplacian_cheb @ V

    return np.concatenate([U_t, V_t])

# Solve the system for no-flux boundary conditions
sol_cheb = solve_ivp(reaction_diffusion_cheb, t_span, UV0_cheb, method='RK45', t_eval=t_eval)
A2 = sol_cheb.y

# Extract solutions
U_sol_no_flux = A2[:(n_chebyshev + 1)**2].reshape((n_chebyshev + 1, n_chebyshev + 1, len(t_eval)))
V_sol_no_flux = A2[(n_chebyshev + 1)**2:].reshape((n_chebyshev + 1, n_chebyshev + 1, len(t_eval)))

print(A2)

[[ 0.70358468  0.27678435 -0.21775865 ... -0.79689015 -0.40972859
   0.07776933]
 [ 0.73241275  0.47188952  0.07344742 ... -0.96577657 -0.78500366
  -0.4261521 ]
 [ 0.81058026  0.37605887 -0.11123233 ... -0.84008598 -0.49565779
  -0.03085913]
 ...
 [ 0.58562756  0.91352592  0.97914313 ... -0.50294695 -0.84298442
  -0.97634716]
 [ 0.6808609   0.87018536  0.97997159 ... -0.16453512 -0.5878894
  -0.88455009]
 [ 0.71061143  0.96093661  0.97601586 ... -0.60413504 -0.91222169
  -0.99697897]]


In [7]:
### ANIMATION ###

# Function to create an animation of U and V
def animate_solution_cheb(U_sol, V_sol, t_eval, title_suffix):
    fig, ax = plt.subplots(1, 2, figsize=(14, 6))

    # Initialize plots
    im_U = ax[0].pcolor(X_cheb, Y_cheb, U_sol[:, :, 0], cmap='RdBu')
    ax[0].set_title(f'U (t={t_eval[0]:.2f}) {title_suffix}')
    ax[0].set_xlabel('x')
    ax[0].set_ylabel('y')
    fig.colorbar(im_U, ax=ax[0], label='U')

    im_V = ax[1].pcolor(X_cheb, Y_cheb, V_sol[:, :, 0], cmap='RdBu')
    ax[1].set_title(f'V (t={t_eval[0]:.2f}) {title_suffix}')
    ax[1].set_xlabel('x')
    fig.colorbar(im_V, ax=ax[1], label='V')

    # Update function for animation
    def update_cheb(frame):
        im_U.set_array(U_sol[:, :, frame].ravel())
        ax[0].set_title(f'U (t={t_eval[frame]:.2f}) {title_suffix}')
        im_V.set_array(V_sol[:, :, frame].ravel())
        ax[1].set_title(f'V (t={t_eval[frame]:.2f}) {title_suffix}')
        return im_U, im_V

    video_duration = t_eval[-1] * 1000 # video duration in miliseconds
    interval = video_duration / len(t_eval)
    ani_cheb = FuncAnimation(fig, update_cheb, frames=len(t_eval), interval=interval, blit=True)
    plt.close(fig)
    return ani_cheb

In [8]:
### ANIMATION ###

t_span_movie = (0, 10)  # time span
t_eval_movie = np.arange(0, 10.1, 0.1)  # evaluation times

# Solve the system
sol_cheb_movie = solve_ivp(reaction_diffusion_cheb, t_span_movie, UV0_cheb, method='RK45', t_eval=t_eval_movie).y

# Extract the solution in Fourier domain (Periodic Boundaries)
U_sol_cheb = sol_cheb_movie[:(n_chebyshev + 1)**2].reshape((n_chebyshev + 1, n_chebyshev + 1, len(t_eval_movie)))
V_sol_cheb = sol_cheb_movie[(n_chebyshev + 1)**2:].reshape((n_chebyshev + 1, n_chebyshev + 1, len(t_eval_movie)))

print(U_sol_cheb.shape)
print(U_sol_cheb.shape)
# Create animation for periodic boundary results
ani_cheb = animate_solution_cheb(U_sol_cheb, V_sol_cheb, t_eval_movie, "(Periodic Boundaries)")
ani_cheb.save(filename="cheb_reaction_diffusion.mp4")

# Display animation
HTML(ani_cheb.to_html5_video())

(31, 31, 101)
(31, 31, 101)
